In [ ]:
!pip install keras-tuner
# Data visualization
import numpy as np
import pandas as pd

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical # Import directly from keras.utils
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt # Now the import should work

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_EYE_INNER",
    "LEFT_EYE",
    "LEFT_EYE_OUTER",
    "RIGHT_EYE_INNER",
    "RIGHT_EYE",
    "RIGHT_EYE_OUTER",
    "LEFT_EAR",
    "RIGHT_EAR",
    "MOUTH_LEFT",
    "MOUTH_RIGHT",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_PINKY",
    "RIGHT_PINKY",
    "LEFT_INDEX",
    "RIGHT_INDEX",
    "LEFT_THUMB",
    "RIGHT_THUMB",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
    "LEFT_HEEL",
    "RIGHT_HEEL",
    "LEFT_FOOT_INDEX",
    "RIGHT_FOOT_INDEX"
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [ ]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")

    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''

    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [ ]:
# load dataset
df = describe_dataset("/content/train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_eye_inner_x', 'left_eye_inner_y', 'left_eye_inner_z', 'left_eye_inner_v', 'left_eye_x', 'left_eye_y', 'left_eye_z', 'left_eye_v', 'left_eye_outer_x', 'left_eye_outer_y', 'left_eye_outer_z', 'left_eye_outer_v', 'right_eye_inner_x', 'right_eye_inner_y', 'right_eye_inner_z', 'right_eye_inner_v', 'right_eye_x', 'right_eye_y', 'right_eye_z', 'right_eye_v', 'right_eye_outer_x', 'right_eye_outer_y', 'right_eye_outer_z', 'right_eye_outer_v', 'left_ear_x', 'left_ear_y', 'left_ear_z', 'left_ear_v', 'right_ear_x', 'right_ear_y', 'right_ear_z', 'right_ear_v', 'mouth_left_x', 'mouth_left_y', 'mouth_left_z', 'mouth_left_v', 'mouth_right_x', 'mouth_right_y', 'mouth_right_z', 'mouth_right_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow

In [ ]:
with open("/content/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [ ]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [ ]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [ ]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [ ]:
def model_3l_builder(hp):
    model = Sequential()
    # Adjust the input dimension to match your data
    model.add(Dense(36, input_dim=132, activation="relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation="softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    model.summary()

    return model

In [ ]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Reloading Tuner from keras_tuner_dir/keras_tuner_demo/tuner0.json


In [ ]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 36)                  │           4,788 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 448)                 │          16,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             898 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,262 (86.96 KB)

 Trainable params: 22,262 (86.96 KB)

 Non-trainable params: 0 (0.00 B)

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 448 units, func:  <function tanh at 0x7fd4082b0940>
Layer-3: 2 units, func:  <function softmax at 0x7fd4082b0550>

Other params:
learning_rate: 0.001
Epoch 1/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.8557 - loss: 0.3011 - val_accuracy: 0.9531 - val_loss: 0.1391
Epoch 2/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9670 - loss: 0.0906 - val_accuracy: 0.9851 - val_loss: 0.0760
Epoch 3/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9798 - loss: 0.0546 - val_accuracy: 0.9926 - val_loss: 0.0250
Epoch 4/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9880 - loss: 0.0398 - val_accuracy: 0.9686 - val_loss: 0.1972
Epoch 5/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9888 - loss: 0.0375 - val_accuracy: 0.9971 - val_loss: 0.0145
Epoch 6/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9920 - loss: 0.0227 - val_accuracy: 0.99

In [ ]:
final_models["3_layers"] = model_3l

In [ ]:
def model_5l_builder(hp):
    model = Sequential()
    # Adjust input dimension to match x_train
    model.add(Dense(36, input_dim=x_train.shape[1], activation="relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation="softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [ ]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 26 Complete [00h 00m 10s]
val_accuracy: 0.9411428570747375

Best val_accuracy So Far: 0.9977142810821533
Total elapsed time: 00h 04m 26s


In [ ]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 480 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-3: 384 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-4: 128 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-5: 2 units, func:  <function softmax at 0x7fd4082b0550>

Other params:
learning_rate: 0.001
Epoch 1/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.8881 - loss: 0.2340 - val_accuracy: 0.9594 - val_loss: 0.0879
Epoch 2/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9747 - loss: 0.0628 - val_accuracy: 0.9811 - val_loss: 0.0368
Epoch 3/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9802 - loss: 0.0499 - val_accuracy: 0.9926 - val_loss: 0.0287


In [ ]:
final_models["5_layers"] = model_5l

In [ ]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim=132, activation="relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 26 Complete [00h 00m 13s]
accuracy: 0.984569251537323

Best accuracy So Far: 0.9964280724525452
Total elapsed time: 00h 09m 22s


In [ ]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 288 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-3: 0 units, func: None
Layer-4: 512 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-5: 0 units, func: None
Layer-6: 512 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-7: 2 units, func:  <function softmax at 0x7fd4082b0550>

Other params:
dropout_1: 0.1
dropout_2: 0.30000000000000004
learning_rate: 0.001
Epoch 1/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8595 - loss: 0.2702 - val_accuracy: 0.9663 - val_loss: 0.0859
Epoch 2/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9720 - loss: 0.0801 - val_accuracy: 0.9709 - val_loss: 0.0715
Epoch 3/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9849 - loss: 0.0466 - val_accuracy: 0.9954 - val_loss: 0.0170


In [ ]:
final_models["7_layers_with_dropout"] = model_7lD

In [ ]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 132, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [ ]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 26 Complete [00h 00m 18s]
val_accuracy: 0.6062856912612915

Best val_accuracy So Far: 0.99657142162323
Total elapsed time: 00h 06m 09s


In [ ]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 288 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-3: 224 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-4: 256 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-5: 480 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-6: 96 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-7: 2 units, func:  <function softmax at 0x7fd4082b0550>

Other params:
learning_rate: 0.0001
Epoch 1/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.8401 - loss: 0.3570 - val_accuracy: 0.9709 - val_loss: 0.0793
Epoch 2/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9787 - loss: 0.0573 - val_accuracy: 0.9863 - val_loss: 0.0391
Epoch 3/100
700/700 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9862 - loss: 0.0343 - val_accuracy: 0.9897 - val_loss: 0.0246


In [ ]:
final_models["7_layers"] = model_7l

In [ ]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 448 units, func:  <function tanh at 0x7fd4082b0940>
Layer-3: 2 units, func:  <function softmax at 0x7fd4082b0550>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 480 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-3: 384 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-4: 128 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-5: 2 units, func:  <function softmax at 0x7fd4082b0550>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-2: 288 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-3: 0 units, func: None
Layer-4: 512 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-5: 0 units, func: None
Layer-6: 512 units, func:  <function relu at 0x7fd466c8f1c0>
Layer-7: 2 units, func:  <function softmax at 0x7fd4082b0550>

7_layers: Descri

In [ ]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])

    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[1.0, 0.993]","[0.988, 1.0]","[0.994, 0.996]","[[681, 8], [0, 1061]]"
1,5_layers,"[0.996, 0.991]","[0.985, 0.997]","[0.991, 0.994]","[[679, 10], [3, 1058]]"
2,3_layers,"[0.986, 0.992]","[0.988, 0.991]","[0.987, 0.992]","[[681, 8], [10, 1051]]"
3,7_layers,"[1.0, 0.983]","[0.974, 1.0]","[0.987, 0.992]","[[671, 18], [0, 1061]]"


In [ ]:
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''

    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)
remove_duplicate_rows("/content/test.csv")

In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = '/content/test.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Remove rows with any missing values
df_cleaned = df.dropna()

# Save the cleaned DataFrame back to a CSV file
cleaned_csv_file_path = '/content/train_cleaned.csv'
df_cleaned.to_csv(cleaned_csv_file_path, index=False)

print(f"Cleaned CSV file has been saved to {cleaned_csv_file_path}.")

Cleaned CSV file has been saved to /content/train_cleaned.csv.


In [ ]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_eye_inner_x', 'left_eye_inner_y', 'left_eye_inner_z', 'left_eye_inner_v', 'left_eye_x', 'left_eye_y', 'left_eye_z', 'left_eye_v', 'left_eye_outer_x', 'left_eye_outer_y', 'left_eye_outer_z', 'left_eye_outer_v', 'right_eye_inner_x', 'right_eye_inner_y', 'right_eye_inner_z', 'right_eye_inner_v', 'right_eye_x', 'right_eye_y', 'right_eye_z', 'right_eye_v', 'right_eye_outer_x', 'right_eye_outer_y', 'right_eye_outer_z', 'right_eye_outer_v', 'left_ear_x', 'left_ear_y', 'left_ear_z', 'left_ear_v', 'right_ear_x', 'right_ear_y', 'right_ear_z', 'right_ear_v', 'mouth_left_x', 'mouth_left_y', 'mouth_left_z', 'mouth_left_v', 'mouth_right_x', 'mouth_right_y', 'mouth_right_z', 'mouth_right_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow

In [ ]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [ ]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False)
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])

    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[1.0, 0.983]","[0.977, 1.0]","[0.989, 0.992]","[[909, 21], [0, 1248]]"
1,7_layers,"[0.999, 0.987]","[0.983, 0.999]","[0.991, 0.993]","[[914, 16], [1, 1247]]"
2,5_layers,"[0.996, 0.987]","[0.982, 0.997]","[0.989, 0.992]","[[913, 17], [4, 1244]]"
3,3_layers,"[0.984, 0.994]","[0.992, 0.988]","[0.988, 0.991]","[[923, 7], [15, 1233]]"


In [ ]:
# Dump the best model to a pickle file
with open("/content/pushup_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers_with_dropout"], f)